In [2]:
import pandas as pd
import pybasilica.run as run
import torch
import pyro
import numpy as np
import seaborn as sns
import sklearn.metrics

%load_ext autoreload
%autoreload 2

/Users/elenab/Library/r-miniconda-arm64/envs/basilica-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
m_g = pd.read_csv("test_datasets/counts_sbs.N150.G3.csv")
m_sbs = m_g.drop(["groups"], axis=1)
g_sbs = m_g["groups"].tolist()
cosmic_sbs = pd.read_csv("test_datasets/COSMIC_filt.csv", index_col=0)

In [4]:
m_g = pd.read_csv("test_datasets/counts_dbs.N150.G3.csv")
m_dbs = m_g.drop(["groups"], axis=1)
g_dbs = m_g["groups"].tolist()
cosmic_dbs = pd.read_csv("test_datasets/COSMIC_dbs.csv", index_col=0) 

In [5]:
sbs_fit = run.fit(
    x=m_sbs, 
    k_list=4, 
    lr=0.005, 
    optim_gamma=0.1,
    n_steps=3000, 
    cluster=6, 
    dirichlet_prior=True,
    beta_fixed=cosmic_sbs.loc[["SBS1","SBS5"]], 
    hyperparameters={"alpha_sigma":.15, "alpha_p_sigma":1., "alpha_p_conc0":0.6, 
                     "alpha_p_conc1":0.6, "alpha_rate":1., "pi_conc0":0.5, "alpha_conc":100,
                     "scale_factor_alpha":5000, "scale_factor_centroid":5000, "scale_tau":0},
    enforce_sparsity = True, 
    reg_weight=0., 
    store_parameters = True, 
    seed=[92],
    nonparametric=True,
    save_all_fits=True
    )

obj_sbs = sbs_fit[0]

N parameters 1284
N parameters 1284
(6, 6)


In [6]:
dbs_fit = run.fit(
    x=m_dbs, 
    k_list=3, 
    lr=0.005, 
    optim_gamma=0.1,
    n_steps=3000, 
    # cluster=6, 
    dirichlet_prior=True,
    beta_fixed=cosmic_dbs.loc[["DBS4"]], 
    hyperparameters={"alpha_sigma":.15, "alpha_p_sigma":1., "alpha_p_conc0":0.6, 
                     "alpha_p_conc1":0.6, "alpha_rate":1., "pi_conc0":0.5, "alpha_conc":100,
                     "scale_factor_alpha":10000, "scale_factor_centroid":1000, "scale_tau":0},
    enforce_sparsity = True, 
    reg_weight=0., 
    store_parameters = True, 
    seed=[92],
    nonparametric=True,
    save_all_fits=True
    )

obj_dbs = dbs_fit[0]

N parameters 834
N parameters 834


In [7]:
alpha_sbs = obj_sbs.params["alpha"]
alpha_dbs = obj_dbs.params["alpha"] 

In [8]:
input = [alpha_sbs, alpha_dbs] 
input_tensor = [torch.tensor(alpha_sbs.values), torch.tensor(alpha_dbs.values)]
max_shape = max([i.shape[1] for i in input_tensor])
# stacked = torch.stack(input_tensor)

In [49]:
a = torch.tensor([[[1,2,3],[4,5,6]]]) 
print(a)
# 2 groups, 3 sigs 

tensor([[[1, 2, 3],
         [4, 5, 6]]])


In [51]:
pyro.distributions.Dirichlet(torch.ones((2,6))).sample((150,))

tensor([[[0.2905, 0.3907, 0.1714, 0.0720, 0.0266, 0.0488],
         [0.0053, 0.2232, 0.0585, 0.0207, 0.4614, 0.2309]],

        [[0.1780, 0.2916, 0.0658, 0.1522, 0.2210, 0.0915],
         [0.0584, 0.0413, 0.0169, 0.0033, 0.2621, 0.6179]],

        [[0.0215, 0.1563, 0.3386, 0.1470, 0.1111, 0.2256],
         [0.2386, 0.1261, 0.0043, 0.0673, 0.4438, 0.1198]],

        ...,

        [[0.0669, 0.0370, 0.0525, 0.1314, 0.5168, 0.1954],
         [0.0033, 0.1725, 0.1658, 0.0444, 0.3571, 0.2569]],

        [[0.0645, 0.0233, 0.1481, 0.1227, 0.2969, 0.3444],
         [0.2417, 0.0440, 0.2539, 0.0484, 0.2422, 0.1698]],

        [[0.4514, 0.2063, 0.1536, 0.0698, 0.0559, 0.0630],
         [0.0974, 0.0358, 0.2181, 0.4319, 0.1759, 0.0409]]])

In [48]:
mixture = run.fit(
    alpha=input, 
    lr=0.005, 
    optim_gamma=0.1,
    n_steps=3000,
    cluster=6, 
    hyperparameters={"alpha_sigma":.15, "alpha_p_sigma":1., "alpha_p_conc0":0.6, 
                     "alpha_p_conc1":0.6, "alpha_rate":1., "pi_conc0":0.5, "alpha_conc":100,
                     "scale_factor_alpha":10000, "scale_factor_centroid":1000, "scale_tau":1},
    store_parameters = True, 
    seed=[92],
    nonparametric=True,
    save_all_fits=True
    )


torch.Size([150, 2, 6])
torch.Size([2, 6, 6])


ValueError: Error while computing log_prob at site 'obs':
Value is not broadcastable with batch_shape+event_shape: torch.Size([150, 2, 6]) vs torch.Size([2, 150, 6]).
    Trace Shapes:            
     Param Sites:            
    Sample Sites:            
        beta dist       5 |  
            value       5 |  
         log_prob       5 |  
     alpha_t dist   2   6 | 6
            value   2   6 | 6
         log_prob   2   6 |  
latent_class dist     150 |  
            value     150 |  
         log_prob     150 |  
         obs dist   2 150 | 6
            value 150   2 | 6

In [39]:
alpha_centr = mixture[0].params["alpha_prior"]
print(alpha_centr) 

     0_SBS1    0_SBS5      0_D1      0_D2      0_D3      0_D4    1_DBS4  \
0  0.486352  0.398079  0.031160  0.083478  0.000550  0.000381  0.206941   
1  0.577721  0.312707  0.010427  0.014838  0.083686  0.000621  0.163690   
2  0.441267  0.407970  0.031460  0.116463  0.001671  0.001169  0.035536   
3  0.486492  0.397324  0.031097  0.084531  0.000309  0.000247  0.473130   
4  0.484065  0.396769  0.031348  0.086922  0.000004  0.000893  0.487679   
5  0.516588  0.375738  0.006624  0.012123  0.088059  0.000868  0.090922   

       1_D1      1_D2      1_D3      1_P0          1_P1  
0  0.608248  0.031189  0.143871  0.009751  2.630111e-37  
1  0.553169  0.187159  0.094616  0.001367  1.600645e-37  
2  0.272111  0.026165  0.566188  0.100000  8.732142e-38  
3  0.384419  0.057587  0.084568  0.000296  2.041855e-36  
4  0.394610  0.031018  0.086675  0.000017  2.619176e-35  
5  0.612470  0.226478  0.068079  0.002051  1.033658e-37  


In [24]:
print(sklearn.metrics.normalized_mutual_info_score(mixture[0].groups, g_sbs)) 
print(sklearn.metrics.normalized_mutual_info_score(mixture[0].groups, g_dbs)) 

0.6116490576138774
0.6188262630406547


In [ ]:
print(obj_sbs.params["scale_factor_centroid"])
print(obj_sbs.params["scale_factor_alpha"]) 

In [ ]:
obj_sbs.params

In [ ]:
obj_sbs.train_params[6]["scale_factor_centroid"]

In [ ]:
obj_sbs.params["pi_conc0"] 

In [ ]:
sns.scatterplot(x=range(len(obj_sbs.likelihoods)), y=obj_sbs.likelihoods) 

In [ ]:
# a / b = 500 
a = 1
b = 1/500
a / b 

In [ ]:
sns.scatterplot(x=range(len(obj_sbs.losses)), y=obj_sbs.losses)

In [ ]:
try: sns.scatterplot(x=range(len(obj_sbs.gradient_norms["scale_factor_centroid_param"])), 
                     y=obj_sbs.gradient_norms["scale_factor_centroid_param"]) 
except: print() 

In [ ]:
try: sns.scatterplot(x=range(len(obj_sbs.gradient_norms["scale_factor_alpha_param"])), 
                     y=obj_sbs.gradient_norms["scale_factor_alpha_param"]) 
except: print() 

In [ ]:
try: sns.scatterplot(x=range(len(obj_sbs.gradient_norms["alpha_prior_param"])), y=obj_sbs.gradient_norms["alpha_prior_param"]) 
except: print() 

In [ ]:
try: sns.scatterplot(x=range(len(obj_sbs.gradient_norms["alpha_prior_param"])), y=obj_sbs.gradient_norms["alpha_prior_param"]) 
except: print() 

In [ ]:
try: sns.scatterplot(x=range(len(obj_sbs.gradient_norms["pi_param"])), y=obj_sbs.gradient_norms["pi_param"]) 
except: print() 

In [ ]:
try: sns.scatterplot(x=range(len(obj_sbs.gradient_norms["pi_conc0_param"])), y=obj_sbs.gradient_norms["pi_conc0_param"]) 
except: print() 

In [ ]:
try: sns.scatterplot(x=range(len(obj_sbs.gradient_norms["alpha"])), y=obj_sbs.gradient_norms["alpha"]) 
except: print() 

In [ ]:
try: sns.scatterplot(x=range(len(obj_sbs.gradient_norms["beta_denovo"])), y=obj_sbs.gradient_norms["beta_denovo"])
except: print()

In [ ]:
pd.DataFrame(np.array(obj_sbs.init_params["alpha_prior_param"]), columns=obj_sbs.params["alpha"].columns).plot.bar(stacked=True, legend=False) 

In [ ]:
try: pd.DataFrame(np.array(obj_sbs.params["alpha_prior"]), columns=obj_sbs.params["alpha_prior"].columns).plot.bar(stacked=True, legend=False) 
except Exception as e: print() 

In [ ]:
try:
    for gid in set(np.array(obj_sbs.groups)):
        tmp = [i for i, v in enumerate(obj_sbs.groups) if v == gid]
        # tmp = [i for i, v in enumerate(obj_sbs.groups) if (v == gid and i in idxs)]
        if len(tmp) == 0: continue
        pd.DataFrame(np.array(obj_sbs.params["alpha"]), columns=obj_sbs.params["alpha"].columns, 
                     index=obj_sbs.params["alpha"].index).iloc[tmp].plot.bar(stacked=True)
except Exception as e:
    print(e)
    obj_sbs.alpha.plot.bar(stacked=True, legend=False) 


In [ ]:
try:
    for sbs in pd.concat((obj_sbs.params["beta_f"], obj_sbs.params["beta_d"])).index:
        pd.concat((obj_sbs.params["beta_f"], obj_sbs.params["beta_d"])).loc[[sbs]].transpose().plot.bar()
except Exception as e:
    print(e)